In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark =SparkContext(conf=conf).getOrCreate()

24/12/05 10:50:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 10:50:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
#여러개의 값을 하나로 축약
# RDD.reduce(<func>)

In [2]:
sample_rdd= spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [3]:
from operator import add

In [4]:
sample_rdd.reduce(add) 

15

# partition 이 나누어져 있을 때 

lambda x,y : (x*2) +y

1: x=1, y=2 >> (1*2)+2 =4
2: x=4, y=3  >> (4*2) +3=1
3: x=11, y=4 >>(11*2) +4 =26

In [5]:
sample_rdd1= spark.parallelize([1,2,3,4]) #여기는 하나의 파티션 
sample_rdd1.reduce(lambda x,y:(x*2)+y)

26

In [6]:
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [8]:
sample_rdd2= spark.parallelize([1,2,3,4],2) #파티션 쪼개기 
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

In [9]:
sample_rdd2.reduce(lambda x,y:(x*2)+y) #파티션 나눴을 떄 
# 4 , 10


18

In [ ]:
#파티션 3개일 때 - 설명 참고 

# x: p1 y:p2 p3:x,y 

In [10]:
#파티션 3개일 때

# x: p1 y:p2 p3:x,y 
sample_rdd3= spark.parallelize([1,2,3,4],3)
sample_rdd3.glom().collect()

[[1], [2], [3, 4]]

In [11]:
sample_rdd3.reduce(lambda x,y:(x*2)+y)

18

In [ ]:
#리듀스 연산은 순서 의존적 
# 각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합 

###fold  연산 활용 
'''

fold( zeroValue, <func> )
    '''
    zeroValue:시작값 , 항등원 * 일 때 1, + 일 경우0 

In [12]:
rdd4= spark.parallelize([2,3,4],4)  #비어있는 값없음 
rdd4.reduce(lambda x,y:x*y)

24

In [13]:
rdd4.fold(1, lambda x,y:x*y)

24

In [17]:
 #오류상황 가정, 빈RDD 예외처리 
rdd5=  spark.parallelize([])  
try:
    rdd5.reduce(lambda a,b:a+b)
except ValueError as e:  
    print(f"Reduce Error : {e}")  #reduce 빈 연산 x

Reduce Error : Can not reduce() empty RDD


In [16]:
rdd5.fold(0,lambda a,b : a+b) #fold 사용 

0

In [ ]:
## reduce(), fold() 비슷한 처리를 합니다.

1. reduce()는 단순한 RDD 축약연산, 비어있지 않은 rdd의 경우 
2. fold()는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음. 

In [18]:
rdd6= spark.parallelize([1,"a",2])
rdd6.collect()

[1, 'a', 2]

In [24]:
try: 
    rdd6.reduce(lambda x,y : str(x)+str(y))
except TypeError as e:
    print(f"Reduce Error : {e}")

In [27]:
try: 
    rdd6.reduce(lambda x,y : x+y)
except Py4JJavaError as e:
    print(f"Reduce Error : {e}")

24/12/05 13:10:56 ERROR Executor: Exception in task 0.0 in stage 23.0 (TID 34)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_3192/3641956346.py", line 2, in <lambda>
TypeError: unsupported operand type(s) for +: 'int' and 'str'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$

NameError: name 'Py4JJavaError' is not defined

In [26]:
# 위 str로 값 안나옴 
from py4j.protocol import Py4JJavaError
try: 
    res=rdd6.reduce(lambda x,y: str(x)+str(y))
    print(type(res),res)
    
except Py4JJavaError as e:
    print(f'reduce 에러가 나네요 : {e}')

ModuleNotFoundError: No module named 'py4j.protoco1'

In [23]:
rdd6.fold("", lambda x,y: str(x)+str(y))

'1a2'

In [ ]:
#groupBy, sggregate()

In [28]:
rdd7= spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [32]:
result = rdd7.groupBy(lambda x: x%2).collect()

In [33]:
sorted([(x,sorted(y)) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

In [34]:
#실행 x
RDD.aggregate(zeroValue, func1_partition, func2_combine)

NameError: name 'RDD' is not defined

In [35]:
func1= lambda x,y : (x[0]+y, x[1]+1)

In [36]:
func2= lambda x,y :(x[0]+y[0], x[1]+y[1])

In [37]:
rdd8 = spark.parallelize( [1,2,3,4],2)

In [38]:
rdd8.aggregate( (0,0), func1, func2)

(10, 4)

In [39]:
rdd8.glom().collect() #값 확인 

[[1, 2], [3, 4]]

###위 시뮬레이션 
p1
x=[1,2], zerovalue=[0,0]
x[0]=0, x[1]=0
x[0]+y = 0+1=1, x[1]+1 = 0+1=1  >> (1,1)
x[0]+y = 1+2=3, x[1]+1 = 1+1=2 >> (3,2)
p2
x=[3,4], zerovalue[0,0]
x[0]=0, x[1]=0
x[0]+y= 0+3=3, x[1]+1= 0+1=1 >> (3,1)
x[0]+y= 3+4=7, x[1]+1= 1+1=2 >> (
p1,p2 
(3,2), (3,1)

In [ ]:
## key-value RDD (paired RDD)

groupByKey(), reduceByKey()

그룹핑 한 후에 특정 트랜스포메이션을 수행, 키가 필요함


In [44]:
rdd9 =spark.parallelize(
    [
        ("짜장면",15),
        ("짬뽕",10),
        ("짜장면",5),
        ("짬뽕",20)
    ])

In [45]:
rdd9_g= rdd9.groupByKey() #rdd를 새로 만드는 것 
rdd9_g

PythonRDD[54] at RDD at PythonRDD.scala:53

In [46]:
#집계형 연산들 
rdd9_g.mapValues(len).collect() #max, min

[('짜장면', 2), ('짬뽕', 2)]

In [47]:
rdd9_g.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [48]:
rdd9_g.mapValues(lambda x:sum(x)/ len(x)).collect()

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [49]:
rdd9.reduceByKey(add).collect() # 바로 연산 적용 가능 , 두단계를 하나로 묶어서 적용해줌/ 

[('짜장면', 20), ('짬뽕', 30)]

In [ ]:
#reduceByKey() > groupByKey () 더 빠름 

In [ ]:
#countByKey()
각 키별로 요소들의 갯수를 카운트 -> action

In [50]:
rdd9.countByKey() #action 이라 결과가 바로 나옴 

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [51]:
#key( ) 키만 가진 RDD -> Transformation

In [53]:
rdd9.keys(). collect()#결과가 나오려면 action 함수 붙여줘야 함 collect

['짜장면', '짬뽕', '짜장면', '짬뽕']

In [54]:
#join() key를 이용한 결합- outer join, inner join 

inner - join :(교집합) 두개의 집합에서 함께 존재하는 요소(key)의 집합 (default 기본값)
outer-  join : 한쪽에있고, 다른쪽에 없을 때 = left, right 

In [56]:
join_rdd1 = spark.parallelize([
    ("a",10),
    ("b",20),
    ("c",30)
])
join_rdd2 = spark.parallelize([
     ("a",20),
     ("b",20),
     ("b",10),
    
        ])
join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]

In [59]:
join_rdd1.leftOuterJoin(join_rdd2).collect() #왼쪽을 기준으로join_rdd1 , 대문자 주의 

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 20))]

In [61]:
join_rdd1.rightOuterJoin(join_rdd2).collect()  #오른쪽을 기준 join_rdd2

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]